### imports

In [2]:
import torch
import numpy as np
from PIL import Image
from sklearn.svm import SVC
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms

import env
from utils import make_numpy_dataset, progress_iter
from utils.pytorch_samplers import StratifiedSampler
from main.optimizers import ClassificationOptimizer
from main.run_small import CNN_Small

## load and preprocess data

In [4]:
X_train = np.load('../data/X_train.npy')
y_train = np.load('../data/y_train.npy')
X_val = np.load('../data/X_val.npy')
y_val = np.load('../data/y_val.npy')

In [7]:
t = transforms.Compose([
    transforms.Lambda(lambda x: Image.fromarray(x)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [9]:
train_dataset = make_numpy_dataset(X_train, y_train, t)
val_dataset = make_numpy_dataset(X_val, y_val, t)

In [14]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=3,
                          sampler=StratifiedSampler(class_vector=y_train,
                                                    batch_size=128))
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=128,
                        shuffle=False,
                        num_workers=3)

## load optimizer and model

In [19]:
optimizer = ClassificationOptimizer(model=CNN_Small(),
                                    optim=torch.optim.SGD, 
                                    optim_params=dict(lr=1e-3, momentum=0.9),
                                    max_epoch=0)
optimizer.load('../models/0.9408-286.ckpt');

## extract features

In [31]:
Z_train = np.zeros((len(X_train), 128), dtype=np.float32)
pos = 0
for X_b, _ in progress_iter(train_loader, verbose=True):
    out = optimizer.model.features(Variable(X_b.cuda())).data.cpu().numpy()
    Z_train[pos:pos + 128] = out[:, :, 0, 0]
    pos += 128

Process Process-24:
Process Process-22:
Traceback (most recent call last):
Process Process-23:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/local/lib/python2.7/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python2.7/dist-packages/torch/utils/data/dataloader.py", line 36,

KeyboardInterrupt: 

In [33]:
Z_val = np.zeros((len(X_val), 128), dtype=np.float32)
pos = 0
for X_b, _ in progress_iter(val_loader, verbose=True):
    out = optimizer.model.features(Variable(X_b.cuda())).data.cpu().numpy()
    Z_train[pos:pos + 128] = out[:, :, 0, 0]
    pos += 128

A Jupyter Widget